In [4]:
## Construct the latent vectors through vae
import numpy as np
import os
import shutil
import random
import torch
import sklearn.utils
from torch.distributions import kl_divergence,Normal
from torch.optim.lr_scheduler import ExponentialLR
from torch.nn import functional as F
from torch import optim

In [5]:
from model import PolyVAE_repara
from torch.utils.data import Dataset, DataLoader, TensorDataset

class MinExponentialLR(ExponentialLR):
    def __init__(self, optimizer, gamma, minimum, last_epoch=-1):
        self.min = minimum
        super(MinExponentialLR, self).__init__(optimizer, gamma, last_epoch=-1)

    def get_lr(self):
        return [
            max(base_lr * self.gamma**self.last_epoch, self.min)
            for base_lr in self.base_lrs
        ]
###############################
# initial parameters
s_dir = ""
batch_size = 64
data_path = [s_dir + "data/poly_train.npy",
             s_dir + "data/poly_validate.npy",
             s_dir + "data/poly_train.npy"]
save_path = s_dir + "model_backup/"
lr = 1e-4
decay = 0.9999
hidden_dims = 1024
z_dims = 1024
vae_beta = 0.1
input_dims = 130
seq_len = 10 * 16
beat_num = 10
tick_num = 16
save_period = 1
##############################

In [6]:
# input data
train_set = np.load(data_path[0], allow_pickle = True)
validate_set = np.load(data_path[1],allow_pickle = True) 

train_x = []
for i,data in enumerate(train_set):
    temp = []
    for d in data["layers"]:
        temp += d
    train_x.append(temp)
train_x = np.array(train_x)
# print(train_x.shape)

validate_x = []
for i,data in enumerate(validate_set):
    temp = []
    for d in data["layers"]:
        temp += d
    validate_x.append(temp)
validate_x = np.array(validate_x)
# print(train_x.shape)
train_x = torch.from_numpy(train_x).long()
validate_x = torch.from_numpy(validate_x).long()

print(train_x.size())
print(validate_x.size())

train_set = TensorDataset(train_x)
validate_set = TensorDataset(validate_x)

train_set = DataLoader(
    dataset = train_set,
    batch_size = batch_size, 
    shuffle = True, 
    num_workers = 8, 
    pin_memory = True, 
    drop_last = True
)
validate_set = DataLoader(
    dataset = validate_set,
    batch_size = batch_size, 
    shuffle = False, 
    num_workers = 8, 
    pin_memory = True, 
    drop_last = True
)



torch.Size([58041, 160])
torch.Size([7578, 160])


In [7]:
# import model
model = PolyVAE_repara(input_dims, hidden_dims, z_dims, seq_len, beat_num, tick_num, 4000)
optimizer = optim.Adam(model.parameters(), lr = lr)

dic = torch.load("model_backup/polyvae-loss_0.08171498498346895_100_90600.pt")

for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
model.load_state_dict(dic)


if decay > 0:
    scheduler = MinExponentialLR(optimizer, gamma = decay, minimum = 1e-5)
if torch.cuda.is_available():
    print('Using: ', torch.cuda.get_device_name(torch.cuda.current_device()))
    model.cuda()
else:
    print('Using: CPU')



Using:  GeForce GTX 1080 Ti


In [ ]:
def loss_function(recon, target, z_mean, z_std, beta):
    CE = F.cross_entropy(recon.view(-1, recon.size(-1)), target, reduction = "mean")
#     rhy_CE = F.nll_loss(recon_rhythm.view(-1, recon_rhythm.size(-1)), target_rhythm, reduction = "mean")
    
    # normal1 =  std_normal(r_dis.mean.size())
    # KLD1 = kl_divergence(r_dis, normal1).mean()
    mean_sq=z_mean * z_mean
    std_sq=z_std * z_std
    KLD1 = 0.5 * torch.mean(mean_sq + std_sq - torch.log(std_sq) - 1)

    max_indices = recon.view(-1, recon.size(-1)).max(-1)[-1]
#     print(max_indices)
    correct = max_indices == target
    acc = torch.sum(correct.float()) / target.size(0)
    return acc, CE + beta * (KLD1)

In [17]:
# decode
model.eval()
device = torch.device(torch.cuda.current_device())
mean_acc = 0.0
ys = []
gds = []
output = []
for i, d in enumerate(validate_set):
    # validate display
    x = gd = d[0]
        
    x = x.to(device = device,non_blocking = True)
    gd = gd.to(device = device,non_blocking = True)
            
    recon, r_dis, iteration, z_mu, z_var = model(x, gd)
        
    acc, _ = loss_function(recon, gd.view(-1), z_mu, z_var, vae_beta)
    mean_acc += acc.item()

#     z = r_dis.rsample()
    pred = recon.argmax(-1)
    output.append({"gd":x.cpu().detach().numpy(), "pred":pred.cpu().detach().numpy(), "acc": acc.item()})
#     print(pred.size())
#     print(v_gd.size())
#     output.append([pred.cpu().detach().numpy(), v_gd.cpu().detach().numpy()])
#     print(v_loss.item())
    print(i, mean_acc / (i + 1))
    if i == 100:
        break
    # np.save("dismeasurevae_gen_exp.npy", output)

0 0.8075195550918579
1 0.7847168147563934
2 0.8089192907015482
3 0.8451904505491257
4 0.8512890815734864
5 0.8511067926883698
6 0.8648856197084699
7 0.8688110485672951
8 0.8665473196241591
9 0.869902354478836
10 0.8755681948228315
11 0.878059908747673
12 0.8714693647164565
13 0.8679059829030719
14 0.873522146542867
15 0.877575695514679
16 0.8826746449751013
17 0.882850491338306
18 0.883254537456914
19 0.8835254043340683
20 0.8834728570211501
21 0.8777121955698187
22 0.8804432905238607
23 0.8781901200612386
24 0.8790117335319519
25 0.8804950576562148
26 0.8823640191996539
27 0.8832833575350898
28 0.8851124870366064
29 0.8856770972410838
30 0.8834079023330442
31 0.8826873917132616
32 0.8820490186864679
33 0.8815918091465446
34 0.8767829367092678
35 0.8762858195437325
36 0.8781487667882765
37 0.8794921997346377
38 0.880013033365592
39 0.8761670053005218
40 0.8773366169231694
41 0.8781505894093287
42 0.878615565078203
43 0.8799183368682861
44 0.8804123388396369
45 0.8815684564735579
46 0.8

In [16]:
np.save("reconstruction_2.npy", output)